<a href="https://colab.research.google.com/github/Araujopri/bot_ipca/blob/main/Notebook_Bot_IPCA_Organizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bot IPCA (SIDRA/IBGE) → Parquet — Notebook organizado

Este notebook consolida os passos em ordem:
1) Setup de pastas (opcionalmente usando Google Drive)  
2) Instalação de dependências  
3) Criação do projeto (`/content/ipca_bot`) e do script `bot_ipca.py` (com **retry/backoff** e **API v2**)  
4) Execução **ao vivo** com fallback para **fixture**  
5) Leitura do Parquet, limpeza e recorte (últimos 24 meses)  
6) Geração de CSV/Parquet finais + README e TXT da Parte 2  


## 1) Setup (pasta de trabalho; monte o Drive se quiser persistir)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

BASE_DIR = '/content/ipca_bot'
import os
os.makedirs(f"{BASE_DIR}/data", exist_ok=True)
os.makedirs(f"{BASE_DIR}/output", exist_ok=True)
print('BASE_DIR:', BASE_DIR)

BASE_DIR: /content/ipca_bot


## 2) Dependências

In [2]:
!pip -q install pandas pyarrow requests

## 3) Criar projeto e o `bot_ipca.py` (API v2 + retry/backoff + fixture)

In [3]:
from pathlib import Path
import json

BASE_DIR = '/content/ipca_bot'

CODE = r'''#!/usr/bin/env python3
"""
Bot de captura do IPCA (SIDRA/IBGE) e gravação em Parquet.
"""
import argparse, json, logging
from pathlib import Path
from typing import List, Dict
import pandas as pd, requests

SIDRA_URLS = [
    "https://apisidra.ibge.gov.br/values/t/1737/n1/all/p/last%20120?formato=json",
    "https://apisidra.ibge.gov.br/values/t/1737/n1/all/p/all?formato=json",
    "https://sidra.ibge.gov.br/Ajax/JSon/Tabela/1/1737?versao=-1",
]

BASE_DIR = Path(__file__).resolve().parent
DATA_DIR = BASE_DIR / "data"
OUT_DIR  = BASE_DIR / "output"
FIXTURE_FILE = DATA_DIR / "sample_ipca.json"

def setup_logging(): logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

def _make_session():
    from requests.adapters import HTTPAdapter
    from urllib3.util.retry import Retry
    retry = Retry(total=5, backoff_factor=1.5, status_forcelist=[429,500,502,503,504], allowed_methods=["GET"], raise_on_status=False)
    s = requests.Session()
    s.headers.update({"User-Agent": "ipca-bot/1.0 (+colab)"})
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    return s

def fetch_json(url: str):
    s = _make_session()
    logging.info("Baixando dados: %s", url)
    r = s.get(url, timeout=30); r.raise_for_status()
    try: return r.json()
    except Exception:
        import json as _j; return _j.loads(r.text)

def fetch_json_with_fallback():
    last_err=None
    for u in SIDRA_URLS:
        try: return fetch_json(u)
        except Exception as e:
            last_err=e; logging.warning("Falha em %s: %s", u, e)
    raise last_err

def load_fixture(path: Path) -> List[Dict]:
    logging.info("Carregando fixture local: %s", path)
    with open(path,"r",encoding="utf-8") as f: return json.load(f)

def normalize_ipca(payload) -> pd.DataFrame:
    if isinstance(payload, list) and len(payload)>1 and isinstance(payload[1], dict):
        regs=[]
        for item in payload[1:]:
            period=str(item.get("D3C") or "")
            ano=int(period[:4]) if period[:4].isdigit() else None
            mes=int(period[4:6]) if len(period)>=6 and period[4:6].isdigit() else None
            v=item.get("V")
            if isinstance(v,str): v=v.replace(",", ".")
            try: valor=float(v)
            except: valor=None
            regs.append({"ano":ano,"mes":mes,"localidade_codigo":item.get("D1C") or "1",
                         "localidade":item.get("D1N") or "Brasil","indice":"IPCA","unidade":"%","valor":valor})
        import pandas as pd
        df=pd.DataFrame(regs).sort_values(["ano","mes"]).reset_index(drop=True)
        return df[["ano","mes","localidade_codigo","localidade","indice","unidade","valor"]]
    import pandas as pd
    try: return pd.json_normalize(payload)
    except Exception: return pd.DataFrame()

def save_parquet(df, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(path, engine="pyarrow", index=False)
    logging.info("Arquivo salvo: %s (linhas=%d, colunas=%d)", path, len(df), df.shape[1])

def main():
    setup_logging()
    import argparse
    p=argparse.ArgumentParser();
    p.add_argument("--live", action="store_true");
    p.add_argument("--out", type=str, default=str(OUT_DIR / "ipca.parquet"))
    args=p.parse_args()
    if args.live:
        try: payload=fetch_json_with_fallback()
        except Exception as e: logging.warning("Falha na rede: %s. Usando fixture.", e); payload=load_fixture(FIXTURE_FILE)
    else:
        payload=load_fixture(FIXTURE_FILE)
    df=normalize_ipca(payload); save_parquet(df, Path(args.out))

if __name__ == "__main__": main()
'''

Path(f"{BASE_DIR}/bot_ipca.py").write_text(CODE, encoding="utf-8")

fixture = [
    {"ano": 2024, "mes": 12, "localidade_codigo": "1", "localidade": "Brasil", "indice": "IPCA", "unidade": "%", "valor": 0.56},
    {"ano": 2025, "mes":  1, "localidade_codigo": "1", "localidade": "Brasil", "indice": "IPCA", "unidade": "%", "valor": 0.42},
]
Path(f"{BASE_DIR}/data").mkdir(parents=True, exist_ok=True)
Path(f"{BASE_DIR}/data/sample_ipca.json").write_text(json.dumps(fixture, ensure_ascii=False, indent=2), encoding="utf-8")
print('Arquivos criados em', BASE_DIR)

Arquivos criados em /content/ipca_bot


## 4) Executar o BOT (live com fallback; se falhar rede, usa fixture)

In [4]:
!python /content/ipca_bot/bot_ipca.py --live

2025-08-26 19:42:12,913 | INFO | Baixando dados: https://apisidra.ibge.gov.br/values/t/1737/n1/all/p/last%20120?formato=json
2025-08-26 19:42:43,205 | WARNING | Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x78ae64631cd0>, 'Connection to apisidra.ibge.gov.br timed out. (connect timeout=30)')': /values/t/1737/n1/all/p/last%20120?formato=json
2025-08-26 19:43:16,603 | WARNING | Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x78ae5f45e060>, 'Connection to apisidra.ibge.gov.br timed out. (connect timeout=30)')': /values/t/1737/n1/all/p/last%20120?formato=json
2025-08-26 19:43:52,748 | WARNING | Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.

## 5) Ler Parquet, limpar e gerar recorte (últimos 24 meses)

In [5]:
import pandas as pd
BASE_DIR = '/content/ipca_bot'
df = pd.read_parquet(f'{BASE_DIR}/output/ipca.parquet')
df = df.dropna(subset=['ano','mes'])
df = df[(df['mes'].between(1,12)) & (df['ano'].between(1990,2100))].copy()
df['data'] = pd.to_datetime(df['ano'].astype(int).astype(str)+'-'+df['mes'].astype(int).astype(str).str.zfill(2)+'-01')
df = df.sort_values('data')
df_24 = df.tail(24).copy()
df.to_parquet(f'{BASE_DIR}/output/ipca_limpo.parquet', engine='pyarrow', index=False)
df_24.to_parquet(f'{BASE_DIR}/output/ipca_ultimos_24m.parquet', engine='pyarrow', index=False)
df_24.to_csv(f'{BASE_DIR}/output/ipca_ultimos_24m.csv', index=False)
df_24.tail()

,ano,mes,localidade_codigo,localidade,indice,unidade,valor,data


## 6) Gerar README e TXT (Parte 2)

In [6]:
BASE_DIR = '/content/ipca_bot'
readme = f"""# Bot IPCA (SIDRA/IBGE) -> Parquet

## Como rodar no Colab
```bash
!pip install pandas pyarrow requests
!python {BASE_DIR}/bot_ipca.py --live
```
Saídas: output/ipca.parquet, ipca_limpo.parquet, ipca_ultimos_24m.parquet, ipca_ultimos_24m.csv
"""
open(f'{BASE_DIR}/README.md','w',encoding='utf-8').write(readme)
texto = """Solução quando não existe link direto:
1) RPA (Playwright/Selenium) headless; menus/logins/cliques.
2) Persistir cookies/tokens; waits explícitos.
3) Interceptar requests (Playwright) para achar endpoint do download.
4) Retry/timeouts/screenshots/logs; captcha com passo assistido.
5) Orquestração: cron/ADF; landing->transform->Parquet.
"""
open(f'{BASE_DIR}/explicacao_sem_link.txt','w',encoding='utf-8').write(texto)
"README.md e explicacao_sem_link.txt gerados."

'README.md e explicacao_sem_link.txt gerados.'

In [7]:
import pandas as pd
df = pd.read_parquet("/content/ipca_bot/output/ipca.parquet")
df.tail()

,ano,mes,localidade_codigo,localidade,indice,unidade,valor
715,2266,None,1,Brasil,IPCA,%,7245.38
716,2266,None,1,Brasil,IPCA,%,7276.54
717,2266,None,1,Brasil,IPCA,%,7295.46
718,2266,None,1,Brasil,IPCA,%,7312.97
719,2266,None,1,Brasil,IPCA,%,7331.98
